## 출력 파서(Output Parsers)

#### 출력파서의 역할
* LLM의 출력을 받아 더 적합한 형식으로 변환
* 구조화된 데이터 생성에 매우 유용
* LangChain 프레임워크에서 다양한 종류의 출력 데이터를 파싱하고 처리


#### 출력파서의 이점 
* 구조화: LLM의 자유 형식 텍스트 출력을 구조화된 데이터로 변환합니다.
* 일관성: 출력 형식을 일관되게 유지하여 후속 처리를 용이하게 합니다.
* 유연성: 다양한 출력 형식(JSON, 리스트, 딕셔너리 등)으로 변환이 가능합니다.

#### 출력파서를 사용하지 않을 때와 사용할 때

* 사용하지 않을 때

```
**중요 내용 추출:**

1. **발신자:** 김철수 (chulsoo.kim@bikecorporation.me)
2. **수신자:** 이은채 (eunchae@teddyinternational.me)
3. **제목:** "ZENESIS" 자전거 유통 협력 및 미팅 일정 제안
4. **요청 사항:**
   - ZENESIS 모델의 상세한 브로슈어 요청 (기술 사양, 배터리 성능, 디자인 정보 포함)
5. **미팅 제안:**
   - 날짜: 다음 주 화요일 (1월 15일)
   - 시간: 오전 10시
   - 장소: 귀사 사무실

6. **발신자 정보:**
   - 김철수, 상무이사, 바이크코퍼레이션
```

* 사용할 때(JSON 형식의 구조화된 답변)

```
{
  "person": "김철수",
  "email": "chulsoo.kim@bikecorporation.me",
  "subject": "\"ZENESIS\" 자전거 유통 협력 및 미팅 일정 제안",
  "summary": "바이크코퍼레이션의 김철수 상무가 테디인터내셔널의 이은채 대리에게 신규 자전거 'ZENESIS' 모델에 대한 브로슈어 요청과 기술 사양, 배터리 성능, 디자인 정보 요청. 또한, 협력 논의를 위해 1월 15일 오전 10시에 미팅 제안.",
  "date": "1월 15일 오전 10시"
}
```


## 01. Pydantic 출력 파서(PydanticOutputParser)

#### PydanticOuputParser
* 언어 모델의 출력을 더 구조화된 정보로 변환 하는 데 도움이 되는 클래스입니다. 단순 텍스트 형태의 응답 대신, 사용자가 필요로 하는 정보를 명확하고 체계적인 형태로 제공
  * get_format_instructions(): 언어 모델이 출력해야 할 정보의 형식을 정의하는 지침(instruction) 을 제공합니다. 예를 들어, 언어 모델이 출력해야 할 데이터의 필드와 그 형태를 설명하는 지침을 문자열로 반환할 수 있습니다. 이때 설정하는 지침(instruction) 의 역할이 매우 중요합니다. 이 지침에 따라 언어 모델은 출력을 구조화하고, 이를 특정 데이터 모델에 맞게 변환할 수 있습니다.

  * parse(): 언어 모델의 출력(문자열로 가정)을 받아들여 이를 특정 구조로 분석하고 변환합니다. Pydantic와 같은 도구를 사용하여, 입력된 문자열을 사전 정의된 스키마에 따라 검증하고, 해당 스키마를 따르는 데이터 구조로 변환합니다.

In [1]:
email_conversation = """From: 김철수 (chulsoo.kim@bikecorporation.me)
To: 이은채 (eunchae@teddyinternational.me)
Subject: "ZENESIS" 자전거 유통 협력 및 미팅 일정 제안

안녕하세요, 이은채 대리님,

저는 바이크코퍼레이션의 김철수 상무입니다. 최근 보도자료를 통해 귀사의 신규 자전거 "ZENESIS"에 대해 알게 되었습니다. 바이크코퍼레이션은 자전거 제조 및 유통 분야에서 혁신과 품질을 선도하는 기업으로, 이 분야에서의 장기적인 경험과 전문성을 가지고 있습니다.

ZENESIS 모델에 대한 상세한 브로슈어를 요청드립니다. 특히 기술 사양, 배터리 성능, 그리고 디자인 측면에 대한 정보가 필요합니다. 이를 통해 저희가 제안할 유통 전략과 마케팅 계획을 보다 구체화할 수 있을 것입니다.

또한, 협력 가능성을 더 깊이 논의하기 위해 다음 주 화요일(1월 15일) 오전 10시에 미팅을 제안합니다. 귀사 사무실에서 만나 이야기를 나눌 수 있을까요?

감사합니다.

김철수
상무이사
바이크코퍼레이션
"""

#### 출력 파서를 사용하지 않는 경우

In [17]:
# 출력 파서를 사용하지 않는 경우
from itertools import chain
from langchain_core.prompts import PromptTemplate
# 실시간 출력을 위한 import
from langchain_teddynote.messages import stream_response
# BaseModel을 사용하면 데이터 검증, 직렬화, 역직렬화 등의 기능을 쉽게 구현
# 필드의 기본 값, 유효성 검사, 설명 등을 지정할 수 있습니다. Field를 사용하여
# 데이터의 구체적인 제약 조건을 설정
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_community.chat_models import ChatOllama
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
load_dotenv()
# llm = ChatOllama(model="llama3.1", temperature=0, max_length=135)
# llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-0125")
llm = ChatOpenAI(temperature=0, model_name="gpt-4o")

prompt =PromptTemplate.from_template(
    "다음의 이메일 내용중 중요한 내용을 추출해 주세요.\n\n{email_conversation}"
)

chain =prompt|llm
# 위의 예시 템플릿내 email_conversation 변수에 삽입
answer= chain.stream({"email_conversation":email_conversation})

output =stream_response(answer, return_output=True)
# 결과값을 보면 모델이 알아서 중요한 내용을 추출하긴 했다. 하지만 업무에 활용할때는 좀 더 구조화된
# 결과물을 선호할수 있다. 

다음은 이메일의 중요한 내용입니다:

1. **요청사항**: ZENESIS 자전거 모델에 대한 상세한 브로슈어 요청 (기술 사양, 배터리 성능, 디자인 정보 포함).
2. **미팅 제안**: 다음 주 화요일(1월 15일) 오전 10시에 미팅 제안 (장소: 귀사 사무실).
3. **목적**: ZENESIS 자전거 유통 협력 논의 및 유통 전략과 마케팅 계획 구체화.

#### Pydantic 스타일로 정의된 클래스를 사용하여 이메일의 정보를 파싱하기
* LLM 경험자로서 이 파서의 가장 큰 장점은 여러질문을 하나로 줄여준다.
* 또한 각각의 자료형까지 선택할수 있는 훌륭한 파서다.

In [3]:
from langchain_core.output_parsers import PydanticOutputParser

class EmailSummary(BaseModel):
    person: str = Field(description="메일을 보낸 사람")
    email: str= Field(description="메일을 보낸 사람의 이메일 주소")
    subject: str = Field(description="메일 제목")
    summary: str = Field(description="메일 본문을 요약한 텍스트")
    date: str = Field(description="메일 본문에 언급된 미팅 날짜와 시간")

parser = PydanticOutputParser(pydantic_object=EmailSummary)

print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"person": {"title": "Person", "description": "\uba54\uc77c\uc744 \ubcf4\ub0b8 \uc0ac\ub78c", "type": "string"}, "email": {"title": "Email", "description": "\uba54\uc77c\uc744 \ubcf4\ub0b8 \uc0ac\ub78c\uc758 \uc774\uba54\uc77c \uc8fc\uc18c", "type": "string"}, "subject": {"title": "Subject", "description": "\uba54\uc77c \uc81c\ubaa9", "type": "string"}, "summary": {"title": "Summary", "description": "\uba54\uc77c \ubcf8\ubb38\uc744 \uc694\uc57d\ud55c \ud14d\uc2a4\ud2b8", "type": "string"}, "date": {"title": "Date", "description"

In [19]:
prompt = PromptTemplate.from_template(
    """
You are a helpful assistant. Please answer the following questions in KOREAN.

QUESTION:
{question}

EMAIL CONVERSATION:
{email_conversation}

FORMAT:
{format}
"""
)

# partial로 일부 인수 format(parser) 삽입
prompt = prompt.partial(format=parser.get_format_instructions())

In [20]:
# chain 을 생성합니다.
chain= prompt | llm

# chain을 실행하고 결과를 출력합니다.
response =chain.stream(
    {
        "email_conversation":email_conversation,
        "question":"이메일 내용중 주요 내용을 추출해 주세요."
    }
)

output =stream_response(response, return_output=True)
# gpt-4o, gpt-3.5-turbo-0125, llama3.1 세개의 모델을 돌려봤는데 gpt-4o만이 제대로 결과를 뽑아냈다.

```json
{
  "person": "김철수",
  "email": "chulsoo.kim@bikecorporation.me",
  "subject": "\"ZENESIS\" 자전거 유통 협력 및 미팅 일정 제안",
  "summary": "바이크코퍼레이션의 김철수 상무가 테디인터내셔널의 이은채 대리에게 신규 자전거 'ZENESIS' 모델에 대한 상세한 브로슈어를 요청하고, 협력 가능성을 논의하기 위해 1월 15일 오전 10시에 미팅을 제안함.",
  "date": "1월 15일 오전 10시"
}
```

In [15]:
# llama3.1 모델은 심지에 결과물이 깨져서 이렇게 후처리 해야했다.
output1=""

import re

def decode_unicode_escapes(input_str):
    # 유니코드 이스케이프 시퀀스를 찾는 정규 표현식
    def decode_match(match):
        return match.group(0).encode('utf-8').decode('unicode_escape')
    
    # 유니코드 이스케이프 시퀀스에 해당하는 부분만 디코딩
    # \\u[0-9a-fA-F]{4}: 유니코드 이스케이프 시퀀스를 찾기 위한 패턴
    # \\u: 유니코드 이스케이프 시퀀스의 시작을 나타냄
    output_str = re.sub(r'(\\u[0-9a-fA-F]{4})+', decode_match, input_str)
    
    return output_str

output1 = decode_unicode_escapes(output)
print(output1)

Here is the output in Korean, formatted as a JSON instance that conforms to the provided schema:

```
{
  "properties": {
    "person": {"title": "메일을 보낸 사람", "description": "", "type": "string"},
    "email": {"title": "메일을 보낸 사람의 이메일 주소", "description": "", "type": "string"},
    "subject": {"title": "메일 제목", "description": "", "type": "string"},
    "summary": {"title": "메일 본문을 요약한 텍스트", "description": "", "type": "string"},
    "date": {"title": "메일 본문에 언급된 미팅 날짜와 시간", "description": "", "type": "string"}
  },
  "required": [
    "person",
    "email",
    "subject",
    "summary",
    "date"
  ],
  "person": "메일을 보낸 사람: 카에이쾔의 시간없안요",
  "email": "카에이쾔의 이메일 주소: eunchae@teddyinternational.me",
  "subject": "메일 제목: ZENESIS",
  "summary": "메일 본문을 요약한 텍스트: 카에이쾔의 시간없안요 메일을 보낸 사람 없안요",
  "date": "메일 본문에 언급된 미팅 날짜와 시간: "
}
```

Note that the `summary` field contains a summary of the email content, which is:

"카에이쾔의 시간없안요 메일을 보낸 사람 없안요: 메일 본문을 요약한 텍스트" which translates to "Summary of the em

### parser 가 추가된 체인 생성

In [21]:
chain = prompt | llm | parser
# chain 을 실행하고 결과를 출력합니다.
response = chain.invoke(
    {
        "email_conversation": email_conversation,
        "question": "이메일 내용중 주요 내용을 추출해 주세요.",
    }
)

# 결과는 EmailSummary 객체 형태로 출력됩니다.
print(response)

person='김철수' email='chulsoo.kim@bikecorporation.me' subject='"ZENESIS" 자전거 유통 협력 및 미팅 일정 제안' summary="바이크코퍼레이션의 김철수 상무가 테디인터내셔널의 이은채 대리에게 신규 자전거 'ZENESIS' 모델에 대한 브로슈어 요청과 유통 협력 논의를 위한 미팅을 제안함. 미팅 일정은 1월 15일 화요일 오전 10시로 제안됨." date='1월 15일 화요일 오전 10시'


### with_structured_output()
* EmailSummary로 개체 하나하나 지정하기 귀찮으면
* .with_structured_output(Pydantic)을 사용하여 출력 파서를 추가하면된다.
* 위에 했던것과 결과물은 같을것이다.

In [22]:
llm_with_structered = ChatOpenAI(
    temperature=0, model_name="gpt-4o"
).with_structured_output(EmailSummary)

# invoke() 함수를 호출하여 결과를 출력합니다.
answer = llm_with_structered.invoke(email_conversation)
answer
# 훨씬 간편하게 파싱을 했고, 결과물이 같은 것을 확인할수 있다.

EmailSummary(person='김철수', email='chulsoo.kim@bikecorporation.me', subject='"ZENESIS" 자전거 유통 협력 및 미팅 일정 제안', summary='바이크코퍼레이션의 김철수 상무가 테디인터내셔널의 이은채 대리에게 ZENESIS 자전거 모델에 대한 브로슈어 요청과 유통 협력 논의를 위한 미팅을 제안합니다. 미팅은 다음 주 화요일(1월 15일) 오전 10시에 제안되었습니다.', date='1월 15일 오전 10시')